# Table of Contents


In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
import ipywidgets as widgets
import sys, os, io, string, shutil, math
from hublib.ui import Submit
from hublib.ui import RunCommand
import hublib.use
%use boost-1.62.0-mpich2-1.3-gnu-4.7.2
%use lammps-31Mar17
#%set_env OMP_NUM_THREADS=4
import numpy as np
import re
import time
from threading import Thread
from io import StringIO
from ipywidgets import Layout, Box, Label, Output
from IPython.display import display,HTML
#sys.path.append('../python/')
#import train as nn
import random
#from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1450px !important; }</style>"))

style = {'description_width': 'initial'}

left_column='340px'
right_column='940px'
app_width='1280px'

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

form_item_layout_slider_text = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

box_layout_image=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
    height='304.5px'
)
box_layout_phisical=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    #width='25%',
    padding='5px',
    #min_height='20px',
    #min_width='200px'
    width=left_column,
    #height='180px'
)
box_layout_computing=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    #height='103px'
)
box_layout_progress=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    ##height='55px'
)

box_layout_output=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=left_column,
    #height='150px'
)
box_layout_about=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    #height='136px'
)
form_item_layout_tab = Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    justify_content='space-between',
    padding='5px'
)

box_layout_tabs=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    padding='5px',
    width=right_column,
    height='100%'
)

button_hide_layout=Layout(
    visibility='hidden', #visible/hidden
    padding='5px',
    
)
button_show_layout=Layout(
    visibility='visible', #visible/hidden
    padding='5px',
    
)


In [ ]:
'''
CSS = """
.output {
    align-items: center;
}
div.output_area {
    width: 100%;
}
div.output_area {
    left-padding: 60px;
}
"""
display(HTML('<style>{}</style>'.format(CSS)))

'''

js_custom="""<script>
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.
</script>
"""

display(HTML(js_custom))

In [ ]:
%%capture
# This hides any output from this cell
%set_env OMP_NUM_THREADS=4

In [ ]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 175px
}
</style>

In [ ]:
# npCharge = widgets.BoundedIntText(
#     value=-1000,
#     min=-1500,
#     max=-500,
#     step=1,
#     description='NP Charge (Q in e)',
#     style=style,
# )

# Added for bnsl but muted for now
e2k2ratio = widgets.Dropdown(
    options={'1:1': 1, '1:4':4},
    value=1,
    description='Ratio (E2:K2)',
    style=style
)


In [ ]:
salt = widgets.BoundedFloatText(
    value=0.175,
    min=0.005,
    max=0.3,
    step=0.005,
    description='Salt Concentration (M):',
    style=style
)


In [ ]:
imageSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=10000,
    description='Images',
    style=style
)

In [ ]:
submit_str = ''
def onToggleBtnChange(b):
    global submit_str
    if clustorMode.value:
        clustorMode.icon = 'check' 
        clustorMode.button_style='success'
        if not submit_str:
            output_image_warning_text.layout.visibility = 'visible'
    else:
        clustorMode.icon = '' 
        clustorMode.button_style=''
        if not submit_str:
            output_image_warning_text.layout.visibility = 'hidden'

'''
clustorMode =widgets.ToggleButton(
    value=False,
    description='Cluster mode',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on but you will loose the access to dynamic simulation snapshot slider.',
    icon='',
    style=style
)
'''
            
clustorMode =widgets.Checkbox(
    value=False,
    description='Cluster mode',
    disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulation will run on a computing cluster when this button is turned on but you will loose the access to dynamic simulation snapshot slider.',
    icon='',
    style=style
)

clustorMode.observe(onToggleBtnChange,'value')

In [ ]:
label_style = "style='background-color: #81c4fd; font-size:500; padding: 2px'"
label_style2 = "style='font-size:350; padding: 2px; font-weight: bold; align: center'"
def make_label(name, label_style, desc="",):
    lval = '<p %s %s>%s</p>' % (desc, label_style, name)
    return widgets.HTML(lval)

plabel = make_label('Input Parameters', label_style, '')
clabel = make_label('Computing Parameters',label_style, '')
olabel = make_label('Output Controllers', label_style, '')
imageSliderabout = make_label('Slide to navigate the simulation snapshots', label_style2, '')

In [ ]:
'''
image_html = widgets.HTML(
    value='<img src="../images/overview.png" style="width: 500px"/>',
)
'''
file = open("../images/overview.png", "rb")
image = file.read()
image=widgets.Image(
    value=image,
    format='jpg',
    width=297,
    height=297,
)

form_items = [
    Box([image], layout=form_item_layout)
]

mainImage = Box(form_items, layout=box_layout_image)

In [ ]:
form_items = [
    Box([plabel], layout=form_item_layout),
    Box([e2k2ratio], layout=form_item_layout),
    Box([salt], layout=form_item_layout)
]

form_items_computing = [
    Box([clabel], layout=form_item_layout),
    Box([clustorMode], layout=form_item_layout)
]

form_items_output = [
    Box([olabel], layout=form_item_layout),
    Box([imageSlider], layout=form_item_layout),
    Box([imageSliderabout], layout=form_item_layout)
]

physical_para = Box(form_items, layout=box_layout_phisical)
computing_para = Box(form_items_computing, layout=box_layout_computing)
output_para = Box(form_items_output, layout=box_layout_output)

In [ ]:
about=widgets.Textarea(
    value='This app simulates assembly of two types of nanoparticles (NPs) into superstructures mediated by linkers. Two types of P22 bacteriophage VLPs (P22-E2 and P22-K2) act as the two distinct NP species with surface charge as the distinguishing feature. Users can input control parameters such as E2 : V2 ratio and ionic strength (from 0.01 M to 0.3 M) to predict the formation of NP aggregates. Outputs are structural information such as simulation snapshots. The NPs are modeled after P22 virus-like particles (VLPs) of diameter 56 nanometers, and linkers represent smaller nanoparticles (dendrimers) of 6.7 nanometer diameter. E2 VLPs(shown in red) have -1500e charge and K2 VLPs(shown in green) have -600e charge. Linker charge is fixed to 45e. Simulations are performed using LAMMPS; pre- and postprocessing are done using C++ codes. Simulation results are tested and validated using SAXS and dynamic light scattering measurements of the VLP aggregates; experiments also guide the model design. After you click Run, the application pre-processes the input parameters and loads the input script in the LAMMPS engine; this may take a couple of minutes after which the output log should appear in the Output text pane. As the output shows, the images will start to populate the "Simulation Snapshot" tab; the slider can be used to navigate through the images. The movie files are available for in the “Download” tab when the simulation ends.',
    placeholder='',
    layout={'height': '119px','width': '100%'},
    disabled=True,
    style = {'font_weight': '150px'}
)

form_about = [
    Box([about], layout=form_item_layout)
]

about_text = Box(form_about, layout=box_layout_about)


In [ ]:
def clear_All():
    
    localOutput.value = ''
    submit_str= ''
    progressBar.value=0
    imageSlider.max=0
    imageSlider.value=0 
    output_image.value=imageEmptyBox
    #movie_download.layout = pair_core_download.layout =  button_hide_layout
    movie_download.layout = button_hide_layout
    clustorMode.disabled = False
    
    if clustorMode.value:
        output_image_warning_text.layout.visibility = 'visible'
    else:
        output_image_warning_text.layout.visibility = 'hidden'
        
    plt.figure('Pair correlation function')
    plt.clf()  # clear old plot
    
    

In [ ]:
def runPreprocessor(e2k2ratio, salt_value):
    preprocessingCMD = "./preprocessor -c "+str(salt_value)+" -r "+str(e2k2ratio)+" > preprocessor.log"  
    
    try:
        !$preprocessingCMD
        
    except  Exception as e:
        localOutput.value = localOutput.value + "Error occured during the execution of preprocessing executable\n"
        sys.stderr.write('Error occured during the execution of preprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1)
    


In [ ]:
runFlag=False
working_dir=""
parameter_append=""
sim_bigin=False
build_Plots=True
simulation_params=""

def callExe(s):
    global runFlag, working_dir, parameter_append, sim_bigin, ionsplot, build_Plots, simulation_params, runName, simulationStepsToatal, e2k2ratio_value
    global submit_str
    if build_Plots:
        showPlotUI()
        build_Plots = False
    clear_All()
    
    #clustorMode.disabled = True
    
    localOutput.value = localOutput.value + "Simulation begins...\n"
     
    e2k2ratio_value = e2k2ratio.value
    salt_value = salt.value
    simulationStepsToatal = 4000000
    total_processors = 64
    walltime = 30
    
    runName='bnsl'
    
    #Adding standard outputs to string to be sent to output widget callback function
    parameter_append= "E2:K2 ratio is "+ str(e2k2ratio_value)+"\n"
    #parameter_append= parameter_append + "Number of Linkers is "+ str(nLigand_value*108)+"\n"
    parameter_append= parameter_append + "Salt Concentration (M) is "+ str(salt_value)+"\n"
    if clustorMode.disabled == False:
        parameter_append= parameter_append + "requested walltime is "+ str(walltime)+" (mins)\n"
        parameter_append= parameter_append + "requested total processors are "+ str(total_processors)+"\n"
    
    
    simulation_params="_%d" % int(e2k2ratio_value)+"_%.2f" % float(salt_value)

    shutil.rmtree('outfiles',True)
    if not os.path.exists('outfiles'):
        os.makedirs('outfiles')
     
    localOutput.value = localOutput.value + "Preprocessing is running...\n"
    
    #preprocessing stage
    runPreprocessor(e2k2ratio_value, salt_value)
    
    localOutput.value = localOutput.value + "Preprocessing completed\n"
        
    localOutput.value = localOutput.value +  parameter_append;
  

    runFlag=True
    sim_bigin=True
    
    #starting a thread for on the fly density plot
    #t2 = Thread(target=iterative_density_plot)
    #t2.start()
    
    try:
        #rname = s.make_rname(e_np_value, e_out_value, pion_value, c_out_value, np_radius_value, np_charge_value, sim_steps_value)
        submit_str = ''
        if clustorMode.value:
            submit_str = '--venue standby@brown -n '+str(total_processors)+' -N 16'+' -w '+str(walltime)+' -e OMP_NUM_THREADS=1'+' --tailStdout 1000 ' +'--inputfile '+'infiles '+'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps'
            #submit_str = '-n '+str(total_processors)+' -w '+str(walltime)+' -N 20'+' -e OMP_NUM_THREADS=1'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps' 
            #submit_str = '--venue standby@rice ' + '-n 32 -w 60 -N 16'+' --tailStdout 1000 ' +'--inputfile '+'outfiles '+'lammps-31Mar17-parallel -in in.lammps'
            localOutput.value = localOutput.value + "LAMMPS input script designed and simulation submitted to the cluster, waiting for response...\n"
        else:
            submit_str = '--local stdbuf -o0 -e0 lmp_serial -in in.lammps'
            localOutput.value = localOutput.value + "LAMMPS input script designed and simulation running...\n"

        s.run(runName, submit_str)
        
        time.sleep(1)
        
    except  Exception as e:
        sys.stderr.write('Error occured during the execution of Lammps program')
        sys.stderr.write(str(e))
        sys.exit(1)
    
    runFlag = False
    #t2.join()

In [ ]:
def doneFunc(s, rdir):
    global submit_str    
    #this is a condtional variable for few things
    submit_str = ''
    
    progressBar.value=int(100)
    
    localOutput.value = localOutput.value + "LAMMPS simulation finished\n" 
    
    submit_btn.but.disabled = True
    clustorMode.disabled = False
    
    localOutput.value = localOutput.value + "Postprocessing is running...\n"
    
    # Determine the number of particles of type1 based on e2k2ratio selected by user
    if e2k2ratio.value == 4:
        num_particles = 40
    else:
        num_particles = 50
    
    #postprocessing stage
    postprocessingCMD = "./postprocessor -x "+str(e2k2ratio.value)+" -N 2001 -e "+str(num_particles)+" > postprocessor.log"
    
    try:
        !$postprocessingCMD
        
    except  Exception as e:
        localOutput.value = localOutput.value + "Error occured during the execution of postprocessing executable\n"
        sys.stderr.write('Error occured during the execution of postprocessing executable')
        sys.stderr.write(str(e))
        sys.exit(1)
        
    localOutput.value = localOutput.value + "Postprocessing is finished\n"
    
    #  Plotting the result
    plot_data()
    
    localOutput.value = localOutput.value + "Simulation ended\n"
    
    submit_btn.but.disabled = False
    
    #image slider fully ready
    imageSlider.max=int(simulationStepsToatal/imageSlider.step)*imageSlider.step
    imageSlider.value=int(simulationStepsToatal/imageSlider.step)*imageSlider.step
    
    movie_download.layout = pair_core_download.layout = button_show_layout
    

In [ ]:
# after the run finished, optionally do something
# with the output
#def plot_data(s, rdir):
def plot_data(): 
    
    global build_Plots
    if build_Plots:
        showPlotUI()
        build_Plots = False 
    
    #We ignore rdir because submit puts non-parametric runs in the current directory
    # When caching is enabled, they get put in a chached directory and rdit is always used. 
    #wait for 2 seconds before plotting errorBars.
    time.sleep(2)  
    plt.figure('Pair correlation function')
    plt.clf()  # clear old plot
    plt.title('NP-NP Pair correlation function')
    plt.xlabel('Distance (in units of NP diameter)', fontsize=10)
    plt.ylabel('g(r)', fontsize=10)
    
    # our simulation writes to outfiles -> density_profile.dat
    try:
        density_profile = np.loadtxt(working_dir + 'outfiles/gr_EE_dr=0.005.out')
    except:
        sys.stderr.write('Can not find the pair correlation results file')
        sys.exit(1)
    #print(density_profile)    
    x = density_profile[:,0]
    # droping data above 4.3 in x dim.
    #x = numpy.select([ x < 4.3 ],x)
    y = density_profile[:x.size,1]
    #yrr = density_profile[:,2]
    #plt.errorbar(x,y,xerr=0.0, yerr=yrr, c='b', ecolor='r', capsize=2)
    plt.xlim([0, 5])
    plt.plot(x,y)
    plt.show()
    

In [ ]:
in_wrapper_progress = False
saved_data = ""
saved_data_enery = ""
saved_data_ions_pos = ""
stepNumber = 0

def stdoutput_handler(buffer):
    global in_wrapper_progress, saved_data, parameter_append, sim_bigin, stepNumber, saved_data_enery, saved_data_ions_pos
    output = []

    #return buffer

    if sim_bigin:
        #output.append(parameter_append)
        sim_bigin = False
   
    data = buffer.split('\n')
    
    last = len(data) - 1
    
    for i, line in enumerate(data):
        
        output.append(line)
        
        if line.startswith('Loop time of '):
            in_wrapper_progress = False
            continue
        
        if in_wrapper_progress:
            number = line.split()
            if number:
                if number[0].isdigit():
                    #print(number[0])
                    percent = float(number[0])/simulationStepsToatal*100
                    progressBar.value=int(percent)
                    # This adjust the image slider dynamically
                    # dynamic update only happens for local VM run
                    if submit_str.startswith('--local '):
                        if(int(number[0])%imageSlider.step == 0):
                            imageSlider.max=number[0]
                            imageSlider.value=number[0]            
            continue
        #Step Temp TotEng KinEng PotEng Volume Press
        if line.startswith('Step Temp TotEng KinEng'):
            in_wrapper_progress = True
            
            
    reduced_buffer = '\n'.join(output)
    
    #appending new line at the end
    if reduced_buffer:
        if reduced_buffer[-1] != '\n':
            reduced_buffer = reduced_buffer + '\n'
    
    
    return reduced_buffer

In [ ]:
#button_style = "style='background-color: #81c4fd; font-size:500; padding: 4px; -webkit-appearance: button; -moz-appearance: button; appearance: button; text-decoration: none; color: initial;'"

def make_downloadBtn(downFilename="", downloadURL="", displayText=""):
    bval = "<a download='%s' href='%s'><button class='p-Widget jupyter-widgets jupyter-button widget-button mod-info'>%s</button></a>" % (downFilename, downloadURL, displayText)
    return widgets.HTML(bval)

#from IPython.display import FileLink, FileLinks
#FileLinks('outfiles')
#FileLinks('data')

In [ ]:
%%capture

tab_layout=Layout(
    width='100%',
    height='588px',
    justify_content='space-between',
)

# we want the plot to go here, so create an empty one initially
#create tabs


figsize_tup=(8.60, 5.05)

plt.ioff() #LINE ADDED, Interactive plot blur fix

figure_density_pos = plt.figure(num='Pair correlation function', figsize=figsize_tup)
out_desnity_pos = Output(layout=tab_layout)

plt.ion() #LINE ADDED

file = open("../images/image.emptybox.jpg", "rb")
imageEmptyBox = file.read()

output_image=widgets.Image(
    format='jpg',
    width=584,
    height=584,
    value=imageEmptyBox
)
#out_output_image = Output(layout=tab_layout)


output_image_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    margin = '0px 0px 0px 130px' #top, right, bottom and left
)

box_layout_warning_text=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 1px',
    align_items='stretch',
    padding='5px',
    width='140px',
    visibility = 'hidden'
    #height='136px'
)

#label_style3 = "style='background-color: #eeeeee; color: #388e3c; font-size:350; padding: 2px; font-weight: bold; align: center'"
label_style3 = "style='background-color: #eeeeee; font-size:350; padding: 2px; font-weight: bold; align: center'"
output_image_warning_text = make_label('Cluster mode is on! You will lose access to the dynamic simulation snapshot data. Snapshot data will be available at the end of simulation.', label_style3, '')

form_output_image_sec = [
    Box([output_image], layout=output_image_layout),
    Box([output_image_warning_text], layout=box_layout_warning_text)
]
output_image_section = Box(form_output_image_sec, layout=tab_layout)

#data download tab
movie_download = make_downloadBtn(downFilename="movie.xyz", downloadURL="outfiles/dump.melt", displayText="Movie")
pair_core_download = make_downloadBtn(downFilename="Pair_correlation_data.out", downloadURL="outfiles/gr_EE_dr=0.005.out", displayText="Pair correlation data")

movie_download.layout = pair_core_download.layout =  button_hide_layout

box_layout_outputs=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    padding='5px'
)
form_output_content = [
    Box([movie_download], layout=box_layout_outputs),
    Box([pair_core_download], layout=box_layout_outputs)]

downloadMovieSec = Box(form_output_content, layout=tab_layout)



In [ ]:
def onSliderChange(b):
    
    value = imageSlider.value
    valueText = str(value)
    if value==0:
        valueText = "000"
    
    fileName="image."+valueText+".jpg"
    fileName="outfiles/"+fileName;
    try:
        file = open(fileName, "rb")
        selectedImage = file.read()
        output_image.value=selectedImage
    except:
        sys.stderr.write('Image file is not there yet.')
       
    
imageSlider.observe(onSliderChange, names='value')

In [ ]:
#tabChange detection and plot data incase automatically not drawn
def onTabChange(b):
    global build_Plots
    if os.path.isfile('outfiles/gr_EE_dr=0.005.out') and not build_Plots:
        plot_data()

In [ ]:
tab_contents = ['Pair Correlation', 'Simulation Snapshot', 'Downloads']
children = [out_desnity_pos, output_image_section, downloadMovieSec]
tabSpace = widgets.Tab()
tabSpace.children = children
for i in range(len(children)):
    tabSpace.set_title(i, str(tab_contents[i]))
tabSpace.selected_index = 1

tabSpace.observe(onTabChange, names='selected_index')

form_about = [
    Box([tabSpace], layout=form_item_layout_tab)
]

tab = Box(form_about, layout=box_layout_tabs)

In [ ]:
def showPlotUI():
    with out_desnity_pos:
        display(figure_density_pos, layout=tab_layout)
    #with out_output_image:
    #    display(output_image, layout=tab_layout)

In [ ]:
form_item_layout_output = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

form_item_layout_run_btn = Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    padding='4px'
)

#Submit(start_func=callExe, done_func=plot_data, cachename='SubmitTest1')
submit_btn=Submit(start_func=callExe, done_func=doneFunc, outcb=stdoutput_handler)

submit_btn.acc.set_title(0, "LAMMPS Output")
submit_btn.w.layout = form_item_layout_run_btn
#submit_btn.acc.layout = form_item_layout_output
localOutput = widgets.Textarea(layout={'width': '100%', 'height': '250px'})
localOutputWidget = widgets.Accordion(children=[localOutput],layout=form_item_layout_output)
localOutputWidget.set_title(0, "Output")
localOutputWidget.selected_index = None
localOutput.value = ""


In [ ]:
progressBar=widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout={'width': '100%'},
    style=style
)

form_items_progress = [
    Box([progressBar], layout=form_item_layout)
]

progressBar_Ele = Box(form_items_progress, layout=box_layout_progress)

In [ ]:
app_height='840px'

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    padding='5px'
)

box_layout_column_left=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=left_column,
    height=app_height
    #height='840px'
)
box_layout_column_right=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=right_column,
    height=app_height
    #height='840px'
)

box_layout=Layout(
    display='flex',
    flex_flow='row',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height=app_height
)

box_layout_rows=Layout(
    display='flex',
    flex_flow='column',
    #border='solid 2px',
    align_items='stretch',
    width=app_width,
    #padding='5px',
    height='1900px',
    #overflow_x='scroll',
    #overflow_y='scroll'
)


gui_left = [
    Box([mainImage], layout=form_item_layout),
    Box([physical_para], layout=form_item_layout),
    Box([computing_para], layout=form_item_layout),
    Box([progressBar_Ele], layout=form_item_layout),
    Box([output_para], layout=form_item_layout)
]

gui_right = [
    Box([about_text], layout=form_item_layout),
    Box([tab], layout=form_item_layout)
]

form_left = Box(gui_left, layout=box_layout_column_left)
form_right = Box(gui_right, layout=box_layout_column_right)

gui_upper = [
    Box([form_left], layout=box_layout_column_left),
    Box([form_right], layout=box_layout_column_right)
]

form = Box(gui_upper, layout=box_layout)

gui = widgets.VBox([form, localOutputWidget, submit_btn.w],layout=box_layout_rows)


In [ ]:
gui